In [1]:
import matplotlib
matplotlib.use('Agg')

import numpy as np
#Pan Sharpening Demo
import skimage
import skimage.io
import skimage.transform

from skimage import data
from skimage.viewer import ImageViewer

In [2]:
#Channels, 2 blue, 3 green, 4 red
def sharpen(root):
    path="/Users/yamaoka/landsat/downloads/"+root+"/clipped/"

    r=skimage.io.imread(path+root+"_B4.TIF", as_grey=True) #Load Red
    g=skimage.io.imread(path+root+"_B3.TIF", as_grey=True) #Load Green
    b=skimage.io.imread(path+root+"_B2.TIF", as_grey=True) #Load Blue
    rgb= np.dstack((r,g,b )) #Combine into correctly ordered stack

    landsat_highres=path+root+"_B8.TIF"
    pan=skimage.io.imread(landsat_highres, as_grey=True)/65535 #Load chromatic, normalize to between 0 and 1

    rgb_big=skimage.transform.resize(rgb, output_shape=(pan.shape[0],pan.shape[1],3), order=3, mode='constant', cval=0.0) #Resize the rgb composite to match the chromatic
    hsv = skimage.color.rgb2hsv(rgb_big) #Conver the rgb to the HSV colorspace
    hsv[...,2]=pan #Poor man's pan sharpening, replace the intensity channel with the observed chromatic image
    rgb_pan = skimage.color.hsv2rgb(hsv) #Convert back to rgb space
    return rgb_pan


In [3]:
%matplotlib inline
landsat1=sharpen(root="LC80270392017030LGN00") #File name of your imagery
#viewer = ImageViewer(landsat1)
#viewer.show()

In [4]:
from scipy import ndimage as ndi
from math import floor

landsat1_dog = landsat1 - ndi.gaussian_filter(landsat1, 15) #Difference of gaussians

In [5]:
stack=np.dstack((landsat1, landsat1_dog))
depth=stack.shape[2]
edge=3
buff=floor(edge/2)
stack_window=skimage.util.view_as_windows(np.pad(stack,pad_width=((buff,buff), (buff,buff),(0,0)), mode ='minimum'), window_shape=(edge,edge,depth), step=1) 
stack_window.shape #3 by 3 moving window around each pixel

stack_flat=stack_window.reshape(-1,edge*edge*depth) ; stack_flat.shape #Flatten to rows

(7639767, 54)

In [6]:
#make labels
building= skimage.io.imread('/Users/yamaoka/Dropbox/Insight/landuse/labels/austin_tx_buildings_labels_allt.TIF', as_grey=True) #Load Buildings
landuse = skimage.io.imread('/Users/yamaoka/Dropbox/Insight/landuse/labels/austin_tx_landuse_labels_noallt.TIF', as_grey=True) #Load Landuse tmp


In [7]:
#viewer = ImageViewer(landuse)
#viewer.show()


In [8]:
ll = landuse.flatten() 
lb = building.flatten() 

print(lb[0])

55537


In [9]:
nodata = 55537

ll[ll != nodata] = 1.
lb[lb != nodata] = 2.

ll[ll == nodata] = 0.
lb[lb == nodata] = 0.

labels = ll+lb
labels[labels==3] = 2. #i know this is stupid need to fix later

In [12]:
print(labels[labels!=0].shape)

(2424080,)


In [13]:
#test_reshape=labels.reshape(landuse.shape)
#print(test_reshape.shape)
#labels_predicted_andknown[labelsfixed>0]=labelsfixed[labelsfixed>0]
#finalcolors=np.array([[0,0,0], #0 unlabeled
#                     [32,66,239], #1 water
#                     [0,102,0], #2 vegitation
#                     [102,51,0], #3 buildings
#                     [0,0,0], #4 rail
#                     [128,128,128]],dtype=np.int64)#5 road
#img = finalcolors[test_reshape]
#outpath = "out/path/here"
#skimage.io.imsave("rf_predictions.png", labels_predicted)
#skimage.io.imsave("/Users/yamaoka/Desktop/test.png", img)

In [14]:
x_tmp=stack_flat
y_tmp=labels

x_tmp = stack_flat[labels>0,...]
y_tmp = labels[labels>0]

y_tmp = y_tmp.astype(int)

from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x_tmp, y_tmp, test_size=0.10, random_state=0)


In [16]:
import pickle
filename='/Users/yamaoka/Dropbox/Insight/landuse/models/rf100_first.sav'
rf=pickle.load(open(filename, 'rb'))

import pickle


In [17]:
rf.score(x_test,y_test)

[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    4.7s
[Parallel(n_jobs=2)]: Done 100 out of 100 | elapsed:   10.3s finished


0.41362908814890598

In [19]:
from sklearn.model_selection import cross_val_score
scores = cross_val_score(rf, x_test, y_test, cv=5)

[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:  1.1min
[Parallel(n_jobs=2)]: Done 100 out of 100 | elapsed:  2.4min finished
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.5s
[Parallel(n_jobs=2)]: Done 100 out of 100 | elapsed:    1.0s finished
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:  1.1min
[Parallel(n_jobs=2)]: Done 100 out of 100 | elapsed:  2.5min finished
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.4s
[Parallel(n_jobs=2)]: Done 100 out of 100 | elapsed:    1.0s finished
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:  1.1min
[Parallel(n_jobs=2)]: Done 100 out of 100 | elapsed:  2.4min finished
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.4s
[Parallel(n_jobs=2)]: Done 100 out of 100 | elapsed:    1.0s finished
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:  1.1min
[Parallel(n_jobs=2)]: Done 100 out of 100 | elapsed:  2.3min finished
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.5s
[Parallel(n_jobs=2)]: 

In [20]:
print(scores)


[ 0.78187781  0.77773194  0.77979456  0.78125451  0.78086261]


In [109]:
labels_predicted=predictions.reshape(labelsfixed.shape)
#labels_predicted_andknown[labelsfixed>0]=labelsfixed[labelsfixed>0]
#inalcolors=np.array([[0,0,0], #0 unlabeled
#                     [32,66,239], #1 water
#                     [0,102,0], #2 vegitation
#                     [102,51,0], #3 buildings
#                     [0,0,0], #4 rail
#                     [128,128,128]])#5 road
#outpath = "out/path/here"
#skimage.io.imsave("rf_predictions.png", labels_predicted)
#skimage.io.imsave("rf_predictions_andknown.png", finalcolors[labels_predicted_andknown])testcolors=np.array([0,0.8,0.2]) 
skimage.io.imsave("rf_predictions.png", testcolors[labels_predicted],)
skimage.io.imsave("dog.png", landsat1_dog)

/Users/yamaoka/anaconda/envs/py35/lib/python3.5/site-packages/skimage/util/dtype.py:110: UserWarning: Possible precision loss when converting from float64 to uint16
  "%s to %s" % (dtypeobj_in, dtypeobj))
